In [94]:
  import os
import numpy as np
from matplotlib import pyplot
from scipy import optimize
import copy

%matplotlib inline

In [95]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [96]:
columa_y = int(input("¿En que columna del (0 a N) esta Y? ")) #0
data = np.loadtxt("/content/gdrive/MyDrive/Segundo_parcial_IA/datos_caballo.txt", delimiter=',')

print(data,data.shape)
print()
X = copy.deepcopy(np.delete(data, columa_y, axis=1)) 
y = copy.deepcopy(data[:,columa_y])
#print(X,X.shape)
print()
#print(y,y.shape)

y = y.astype(int) # convierte a enteros 

y[y == 1] = int('0')
y[y == 2] = int('1')
y[y == 3] = int('2')

# X= X[150:,:]
Xp = X[:150,:]

# y = y[150:]
yp = y[:150]

print(X, X.shape)
print(y, y.shape)

¿En que columna del (0 a N) esta Y? 22
[[2.00000e+00 1.00000e+00 5.30101e+05 ... 0.00000e+00 0.00000e+00
  2.00000e+00]
 [1.00000e+00 1.00000e+00 5.34817e+05 ... 0.00000e+00 0.00000e+00
  2.00000e+00]
 [2.00000e+00 1.00000e+00 5.30334e+05 ... 0.00000e+00 0.00000e+00
  1.00000e+00]
 ...
 [1.00000e+00 1.00000e+00 5.29386e+05 ... 0.00000e+00 0.00000e+00
  2.00000e+00]
 [1.00000e+00 1.00000e+00 5.30612e+05 ... 0.00000e+00 0.00000e+00
  1.00000e+00]
 [1.00000e+00 1.00000e+00 5.34618e+05 ... 0.00000e+00 0.00000e+00
  2.00000e+00]] (300, 28)


[[2.00000e+00 1.00000e+00 5.30101e+05 ... 0.00000e+00 0.00000e+00
  2.00000e+00]
 [1.00000e+00 1.00000e+00 5.34817e+05 ... 0.00000e+00 0.00000e+00
  2.00000e+00]
 [2.00000e+00 1.00000e+00 5.30334e+05 ... 0.00000e+00 0.00000e+00
  1.00000e+00]
 ...
 [1.00000e+00 1.00000e+00 5.29386e+05 ... 0.00000e+00 0.00000e+00
  2.00000e+00]
 [1.00000e+00 1.00000e+00 5.30612e+05 ... 0.00000e+00 0.00000e+00
  1.00000e+00]
 [1.00000e+00 1.00000e+00 5.34618e+05 ... 0.000

In [ ]:
input_layer_size  = 27  # capa de entrada 
hidden_layer_size = 28  # Neuronas de capa oculta
num_labels = 3          # clases 3 (0 -> 2)

pesos = {}
pesos['Theta1'] = np.random.rand(hidden_layer_size, input_layer_size + 1)
pesos['Theta2'] = np.random.rand(num_labels, hidden_layer_size + 1)

Theta1, Theta2 = pesos['Theta1'], pesos['Theta2']

print(Theta1,Theta1.shape)
print(Theta2,Theta2.shape)

In [98]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

def sigmoidGradient(z): 
    g = np.zeros(z.shape)
    g = sigmoid(z) * (1 - sigmoid(z)) #  Derivada de Sigmoid
    return g

In [99]:
def nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_=0.01):
    Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                        (hidden_layer_size, (input_layer_size + 1)))

    Theta2 = np.reshape(nn_params[(hidden_layer_size * (input_layer_size + 1)):],
                        (num_labels, (hidden_layer_size + 1)))

    m = y.size     
    J = 0
    Theta1_grad = np.zeros(Theta1.shape)
    Theta2_grad = np.zeros(Theta2.shape)

    a1 = np.concatenate([np.ones((m, 1)), X], axis=1)    
    a2 = sigmoid(a1.dot(Theta1.T))
    a2 = np.concatenate([np.ones((a2.shape[0], 1)), a2], axis=1)   
    a3 = sigmoid(a2.dot(Theta2.T))
 
    y_matrix = y.reshape(-1)
    # print(y.shape)
    y_matrix = np.eye(num_labels)[y_matrix]#genera una matriz de unos con filas de tamaño (y) y columnas (labels);el 1 se crea en la pocicion y[i] 
    # print(y_matrix)
    temp1 = Theta1
    temp2 = Theta2  
    # Agregar el termino de regularización  
    reg_term = (lambda_ / (2 * m)) * (np.sum(np.square(temp1[:, 1:])) + np.sum(np.square(temp2[:, 1:])))  
    J = (-1 / m) * np.sum((np.log(a3) * y_matrix) + np.log(1 - a3) * (1 - y_matrix)) + reg_term  
    # Backpropogation    
    delta_3 = a3 - y_matrix
    delta_2 = delta_3.dot(Theta2)[:, 1:] * sigmoidGradient(a1.dot(Theta1.T))

    Delta1 = delta_2.T.dot(a1)
    Delta2 = delta_3.T.dot(a2)
    # Agregar regularización al gradiente
    Theta1_grad = (1 / m) * Delta1
    Theta1_grad[:, 1:] = Theta1_grad[:, 1:] + (lambda_ / m) * Theta1[:, 1:]
    
    Theta2_grad = (1 / m) * Delta2
    Theta2_grad[:, 1:] = Theta2_grad[:, 1:] + (lambda_ / m) * Theta2[:, 1:]    
    grad = np.concatenate([Theta1_grad.ravel(), Theta2_grad.ravel()])   
    return J, grad

In [100]:
def randInitializeWeights(L_in, L_out, epsilon_init=0.12):
    W = np.zeros((L_out, 1 + L_in))
    W = np.random.rand(L_out, 1 + L_in) * 2 * epsilon_init - epsilon_init
    return W

In [101]:
print('Inicialización de parámetros de redes neuronales...')
initial_Theta1 = randInitializeWeights(input_layer_size, hidden_layer_size)
initial_Theta2 = randInitializeWeights(hidden_layer_size, num_labels)
# Desenrrollr parametros
initial_nn_params = np.concatenate([initial_Theta1.ravel(), initial_Theta2.ravel()], axis=0)

Inicialización de parámetros de redes neuronales...


In [102]:
options= {'maxiter': 1000}
lambda_ = 1
costFunction = lambda p: nnCostFunction(p, input_layer_size, hidden_layer_size, num_labels, X, y, lambda_)

res = optimize.minimize(costFunction, initial_nn_params, jac=True, method='BFGS', options=options)
nn_params = res.x

Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)], (hidden_layer_size, (input_layer_size + 1)))
Theta2 = np.reshape(nn_params[(hidden_layer_size * (input_layer_size + 1)):], (num_labels, (hidden_layer_size + 1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


In [103]:
def predict(Theta1, Theta2, X):
    m = X.shape[0]
    num_labels = Theta2.shape[0]
    p = np.zeros(m)
    h1 = sigmoid(np.dot(np.concatenate([np.ones((m, 1)), X], axis=1), Theta1.T))
    h2 = sigmoid(np.dot(np.concatenate([np.ones((m, 1)), h1], axis=1), Theta2.T))
    p = np.argmax(h2, axis=1)
    return p

In [104]:
pred = predict(Theta1, Theta2, X[:,:])
print(pred)
print('Precisión del conjunto de entrenamiento: %f' % (np.mean(pred == y[:]) * 100))

[1 2 0 1 1 0 0 1 2 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 1 0 1 2 0 0 0 2 2
 0 0 1 1 0 0 2 2 1 0 0 1 1 0 1 0 0 2 1 0 1 1 1 0 0 1 2 0 0 0 0 0 0 0 0 1 1
 1 2 0 0 1 1 0 0 1 1 0 0 1 0 0 0 0 1 0 1 0 0 0 2 0 0 0 2 0 0 1 0 0 0 2 1 1
 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 2 0 0 1 0 0 1 1 1 0 0 0 0 1 1 0 2 0 0 1
 0 0 0 1 0 0 0 0 0 2 0 0 0 1 0 2 0 2 0 0 1 2 0 1 0 1 0 0 0 1 0 2 1 0 1 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 2 0 1 2 0 1 1 1 0 0 0 0 1 0 0 2 0
 0 0 0 0 1 0 2 0 0 1 0 0 0 1 0 0 2 2 1 0 1 1 0 1 0 1 0 0 2 1 1 0 0 1 2 1 0
 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 2 1 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 1 1 2 0 2
 0 1 0 2]
Precisión del conjunto de entrenamiento: 92.000000


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


In [105]:
pred = predict(Theta1, Theta2, Xp[:,:])
print(pred)
print('Precisión del conjunto de entrenamiento: %f' % (np.mean(pred == yp[:]) * 100))


[1 2 0 1 1 0 0 1 2 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 1 0 1 2 0 0 0 2 2
 0 0 1 1 0 0 2 2 1 0 0 1 1 0 1 0 0 2 1 0 1 1 1 0 0 1 2 0 0 0 0 0 0 0 0 1 1
 1 2 0 0 1 1 0 0 1 1 0 0 1 0 0 0 0 1 0 1 0 0 0 2 0 0 0 2 0 0 1 0 0 0 2 1 1
 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 2 0 0 1 0 0 1 1 1 0 0 0 0 1 1 0 2 0 0 1
 0 0]
Precisión del conjunto de entrenamiento: 92.666667


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
